In [ ]:
import torch
import torch.nn as nn

class EvaluatorLoss(nn.Module):

    def __init__(self, alpha: float = 0.5, beta: float = 0.5):
        super().__init__()
        self.alpha = alpha
        self.beta = beta

    def forward(self, evaluator_outputs, generator_outputs, other_outputs):
        generator_outputs = 1 - generator_outputs
        other_outputs = 1 - other_outputs
        temp = 1e-5
        evaluator_outputs = evaluator_outputs.clamp(min=temp)
        generator_outputs = generator_outputs.clamp(min=temp)
        other_outputs = other_outputs.clamp(min=temp)
        t1, t2, t3 = torch.log(evaluator_outputs), self.alpha * torch.log(generator_outputs), self.beta * torch.log(
            other_outputs)
        result = t1 + t2 + t3
        result = -result
        return result.mean()

In [ ]:
import os
import time
from multiprocessing import cpu_count

from torch import optim
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence
from torch.utils.data import DataLoader
import torch.nn.functional as F

data_folder = '/home/why/image-captioning-bottom-up-top-down/other_dataset'  # folder with data files saved by create_input_files.py
data_name = 'coco_5_cap_per_img_5_min_word_freq'  # base name shared by data files
checkpoint = "BEST_42checkpoint_coco_5_cap_per_img_5_min_word_freq.pth.tar"

def evaluator_main():
    if not os.path.exists("models"):
        os.makedirs("models")
    checkpoint = "BEST_42checkpoint_coco_5_cap_per_img_5_min_word_freq.pth.tar"
    
    word_map_file = os.path.join(data_folder, 'WORDMAP_' + data_name + '.json')
    with open(word_map_file, 'r') as j:
        word_map = json.load(j)
    
    checkpoint = torch.load(checkpoint)
    generator = checkpoint['decoder']
    generator = generator.to(device)
    generator.eval()
    
    
    evaluator = Evaluator( embed_dim = 1024, decoder_dim = 1024, vocab_size = len(word_map),
                           features_dim=2048, dropout=0.5,max_len = 20)
    evaluator = evaluator.to(device)
    
    batch_size = 128
    dataloader = torch.utils.data.DataLoader(CaptionDataset(data_folder, data_name, 'TRAIN'),
                        batch_size=batch_size, shuffle=True, num_workers=1, pin_memory=True)
    
    criterion = EvaluatorLoss().cuda()
    optimizer = optim.Adam(evaluator.parameters(), lr=5e-5, weight_decay=1e-5)
#######################################################################################################    
    
    epoch = 10
    print(f"number of batches = {len(dataloader) // batch_size}")
    print("Begin Training")
    for epoch in range(epochs):
        cap_out = 0
        gen_out = 0
        for i, (imgs, caps, _,oth_caps,_ ) in enumerate(dataloader):
            start = time.time()
            imgs = imgs.to(device)
            caps = caps.to(device)
            caps = generator.embedding(caps)  # (batch_size, max_caption_length, embed_dim)
            oth_caps = oth_caps.to(device)
            oth_caps = generator.embedding(oth_caps) 
            
            generator_outputs = generator.sample(imgs)
            generator_outputs = tran(generator_outputs)
            generator_outputs = generator.embedding(generator_outputs) 

            evaluator_outputs = evaluator(imgs, caps)     #  鉴别器 鉴别  image 正确 caption  
            generator_outputs = evaluator(imgs, generator_outputs)  #  image  生成的虚假caption
            other_outputs = evaluator(imgs, oth_caps)  # image  其他图像的caption
            cap_out = cap_out+evaluator_outputs.sum()
            gen_out = gen_out+generator_outputs.sum()
            
            loss = criterion(evaluator_outputs, generator_outputs, other_outputs)
            
            loss.backward()
            optimizer.step()
            end = time.time()
            if i % 100 == 0:
                print('Epoch: [{0}][{1}/{2}]\t'
                      'Batch Time {batch_time:.3f}\t'
                      'Loss {loss:.4f} '.format(epoch, i, len(dataloader),
                                                batch_time=end - start,loss=loss))
            start = end
        
        evaluator.save()
        cap_ave = cap_out/113287
        gen_ave = gen_out/113287
        print("cap_ave:------",cap_ave)
        print("gen_ave:------",gen_ave)
        if cap_ave>0.5 and gen_ave<0.5:
            break
        print(f"Epoch = {epoch + 1}")
        

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np

import time
import os
from six.moves import cPickle
import traceback

import opts
import models
from dataloader import *
import skimage.io
import eval_utils

/home/why/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
model = models.setup(opt)

NameError: name 'opt' is not defined

In [ ]:
model.load_state_dict(torch.load(opt.model))
model.cuda()
model.eval()
crit = utils.LanguageModelCriterion()

In [ ]:

    

    # Create the Data Loader instance
    if len(opt.image_folder) == 0:
        loader = DataLoader(opt)
    else:
        loader = DataLoaderRaw({'folder_path': opt.image_folder, 
                                'coco_json': opt.coco_json,
                                'batch_size': opt.batch_size,
                                'cnn_model': opt.cnn_model})
    
    beam_size = eval_kwargs.get('beam_size', 1)
    # Make sure in the evaluation mode
    model.eval()

    loader.reset_iterator(split)
    n = 0
    while True:
        data = loader.get_batch('train')
        n = n + loader.batch_size

        if data.get('labels', None) is not None:
            # forward the model to get loss
            tmp = [data['fc_feats'], data['att_feats'], data['labels'], data['masks'], data['att_masks'],data['topic_word'],data['other_caption']]
            tmp = [_.cuda() if _ is not None else _ for _ in tmp]
            fc_feats, att_feats, labels, masks, att_masks, topic_word,other_caption= tmp

        # forward the model to also get generated samples for each image
        # Only leave one feature for each image, in case duplicate sample
        tmp = [data['fc_feats'][np.arange(loader.batch_size) * loader.seq_per_img], 
            data['att_feats'][np.arange(loader.batch_size) * loader.seq_per_img],
            data['topic_word'][np.arange(loader.batch_size) * loader.seq_per_img],
            data['other_caption'][np.arange(loader.batch_size) * loader.seq_per_img],
            data['att_masks'][np.arange(loader.batch_size) * loader.seq_per_img] if data['att_masks'] is not None else None]
        tmp = [_.cuda() if _ is not None else _ for _ in tmp]
        fc_feats, att_feats, topic_word, other_caption,att_masks = tmp
      
        # forward the model to also get generated samples for each image
        with torch.no_grad():
            seq = model(fc_feats, att_feats, topic_word, att_masks, opt=eval_kwargs, mode='sample')[0].data

    return seq


In [ ]:
def add_summary_value(writer, key, value, iteration):
    if writer:
        writer.add_scalar(key, value, iteration)

def train(opt):
    opt.use_fc, opt.use_att = utils.if_use_feat(opt.caption_model)
    if opt.use_box: opt.att_feat_size = opt.att_feat_size + 5

    loader = DataLoader(opt)
    opt.vocab_size = loader.vocab_size
    opt.seq_length = loader.seq_length

    tb_summary_writer = tb and tb.SummaryWriter(opt.checkpoint_path)

    infos = {}
    histories = {}
    with open(os.path.join(opt.start_from, 'infos_'+opt.id+'.pkl'), 'rb') as f:
        infos = utils.pickle_load(f)
        saved_model_opt = infos['opt']
        need_be_same=["caption_model", "rnn_type", "rnn_size", "num_layers"]
        for checkme in need_be_same:
            assert vars(saved_model_opt)[checkme] == vars(opt)[checkme], "Command line argument and saved model disagree on '%s' " % checkme

    if os.path.isfile(os.path.join(opt.start_from, 'histories_'+opt.id+'.pkl')):
        with open(os.path.join(opt.start_from, 'histories_'+opt.id+'.pkl'), 'rb') as f:
            histories = utils.pickle_load(f)
    infos['opt'] = opt

    iteration = infos.get('iter', 0)
    epoch = infos.get('epoch', 0)

    val_result_history = histories.get('val_result_history', {})
    loss_history = histories.get('loss_history', {})
    lr_history = histories.get('lr_history', {})
    ss_prob_history = histories.get('ss_prob_history', {})

    loader.iterators = infos.get('iterators', loader.iterators)
    loader.split_ix = infos.get('split_ix', loader.split_ix)
    if opt.load_best_score == 1:
        best_val_score = infos.get('best_val_score', None)

###################################################################################################        

    model = models.setup(opt).cuda()
    lw_model = LossWrapper(model, opt)
    dp_lw_model = torch.nn.DataParallel(lw_model)
    
    epoch_done = True
    # Assure in training mode
    dp_lw_model.eval()

    if opt.noamopt:
        assert opt.caption_model == 'transformer', 'noamopt can only work with transformer'
        optimizer = utils.get_std_opt(model, factor=opt.noamopt_factor, warmup=opt.noamopt_warmup)
        optimizer._step = iteration
    elif opt.reduce_on_plateau:
        optimizer = utils.build_optimizer(model.parameters(), opt)
        optimizer = utils.ReduceLROnPlateau(optimizer, factor=0.5, patience=3)
    else:
        optimizer = utils.build_optimizer(model.parameters(), opt)
    # Load the optimizer
    if vars(opt).get('start_from', None) is not None and os.path.isfile(os.path.join(opt.start_from,"optimizer.pth")):
        optimizer.load_state_dict(torch.load(os.path.join(opt.start_from, 'optimizer.pth')))

    while True:
        sc_flag = True
        init_scorer(opt.cached_tokens)
        start = time.time()
        # Load data from train split (0)
        data = loader.get_batch('train')
        print('Read data:', time.time() - start)

        torch.cuda.synchronize()
        start = time.time()

        tmp = [data['fc_feats'], data['att_feats'], data['labels'], data['masks'], data['att_masks'],data['topic_word'],data['other_caption']]
        tmp = [_ if _ is None else _.cuda() for _ in tmp]
        fc_feats, att_feats, labels, masks, att_masks, topic_word,other_caption= tmp

        optimizer.zero_grad()
        model_out = dp_lw_model(fc_feats, att_feats, labels, masks,topic_word, att_masks, data['gts'], torch.arange(0, len(data['gts'])), sc_flag)
        
        
        loss = model_out['loss'].mean()

        loss.backward()
        utils.clip_gradient(optimizer, opt.grad_clip)
        optimizer.step()
        train_loss = loss.item()
        torch.cuda.synchronize()
        end = time.time()
        print("iter {} (epoch {}), avg_reward = {:.3f}, time/batch = {:.3f}" \
                .format(iteration, epoch, model_out['reward'].mean(), end - start))

        # Update the iteration and epoch
        iteration += 1
        if data['bounds']['wrapped']:
            epoch += 1
            epoch_done = True

        # Write the training loss summary
        if (iteration % opt.losses_log_every == 0):
            add_summary_value(tb_summary_writer, 'train_loss', train_loss, iteration)
            if opt.noamopt:
                opt.current_lr = optimizer.rate()
            elif opt.reduce_on_plateau:
                opt.current_lr = optimizer.current_lr
            add_summary_value(tb_summary_writer, 'learning_rate', opt.current_lr, iteration)
            add_summary_value(tb_summary_writer, 'scheduled_sampling_prob', model.ss_prob, iteration)
            if sc_flag:
                add_summary_value(tb_summary_writer, 'avg_reward', model_out['reward'].mean(), iteration)

            loss_history[iteration] = train_loss if not sc_flag else model_out['reward'].mean()
            lr_history[iteration] = opt.current_lr
            ss_prob_history[iteration] = model.ss_prob

        # update infos
        infos['iter'] = iteration
        infos['epoch'] = epoch
        infos['iterators'] = loader.iterators
        infos['split_ix'] = loader.split_ix

        
###################################################################################################################      
        
        # make evaluation on validation set, and save model
        eval_kwargs = {'split': 'val',
                        'dataset': opt.input_json}
        eval_kwargs.update(vars(opt))
        val_loss, predictions, lang_stats = eval_utils.eval_split(
            dp_model, lw_model.crit, loader, eval_kwargs)
        
 
    # Stop if reaching max epochs
    if epoch >= opt.max_epochs and opt.max_epochs != -1:
        break

In [14]:
import torch
import numpy as np

In [27]:
aa = torch.Tensor(5,1)

In [30]:
aa

tensor([[1.0877e+11],
        [4.5803e-41],
        [9.4711e+03],
        [3.0638e-41],
        [4.4842e-44]])

In [33]:
bb = aa.repeat(1,5)

In [34]:
bb

tensor([[1.0877e+11, 1.0877e+11, 1.0877e+11, 1.0877e+11, 1.0877e+11],
        [4.5803e-41, 4.5803e-41, 4.5803e-41, 4.5803e-41, 4.5803e-41],
        [9.4711e+03, 9.4711e+03, 9.4711e+03, 9.4711e+03, 9.4711e+03],
        [3.0638e-41, 3.0638e-41, 3.0638e-41, 3.0638e-41, 3.0638e-41],
        [4.4842e-44, 4.4842e-44, 4.4842e-44, 4.4842e-44, 4.4842e-44]])

In [35]:
bb = bb.view(25,1)

In [36]:
bb

tensor([[1.0877e+11],
        [1.0877e+11],
        [1.0877e+11],
        [1.0877e+11],
        [1.0877e+11],
        [4.5803e-41],
        [4.5803e-41],
        [4.5803e-41],
        [4.5803e-41],
        [4.5803e-41],
        [9.4711e+03],
        [9.4711e+03],
        [9.4711e+03],
        [9.4711e+03],
        [9.4711e+03],
        [3.0638e-41],
        [3.0638e-41],
        [3.0638e-41],
        [3.0638e-41],
        [3.0638e-41],
        [4.4842e-44],
        [4.4842e-44],
        [4.4842e-44],
        [4.4842e-44],
        [4.4842e-44]])